In [6]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import spawn_camera, spawn_vehicle, preprocess_image, spawn_camera_depth, riconosci_parcheggio1

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [57]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-1, -40, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True

In [58]:
def camera_callback(image):
    global run
    prediction=riconosci_parcheggio1(preprocess_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))))
    if prediction > 0.65:
        run = False
camera_rigth.listen(lambda image: camera_callback(image))

In [59]:
control = carla.VehicleControl()
control.steer = 0.0 
control.throttle = 0.5 
control.brake = 0.0  

while run:
        vehicle.apply_control(control)
        time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(1.1)

control.throttle = 0.5 
control.brake = 0.0 
i = 0 
while i<21:
        vehicle.apply_control(control)
        i=i+1
        time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(0.1)

i = 0 
while i<83:
        control = carla.VehicleControl(throttle=0.3, steer=0.8, brake=0.0, reverse=True)
        vehicle.apply_control(control)
        i=i+1
        time.sleep(0.1)
control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0, reverse=False)
vehicle.apply_control(control)
time.sleep(4.1)
camera_retro.destroy()
camera_rigth.destroy()
vehicle.destroy()

True

In [19]:
import carla
import time

# Connetti al simulatore CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# Ottieni il mondo
world = client.get_world()

# Crea un veicolo (ad esempio, una macchina)
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')  # Sostituisci con il modello di veicolo desiderato
spawn_point = carla.Transform(carla.Location(-1, -20, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Imposta il controllo per far muovere il veicolo in avanti
def move_vehicle_forward(distance):
    # Calcola il tempo necessario in base alla velocità del veicolo
    velocity = 10  # Imposta la velocità in m/s (esempio 10 m/s)
    time_to_travel = distance / velocity  # Tempo in secondi

    # Imposta il controllo
    control = carla.VehicleControl()
    control.throttle = 0.50  # Accelera al massimo
    control.steering = 0.0  # Mantieni la direzione diritta

    vehicle.apply_control(control)

    # Mantieni la velocità per il tempo calcolato
    time.sleep(time_to_travel)

    # Arresta il veicolo dopo aver percorso la distanza
    control.throttle = 0.0
    vehicle.apply_control(control)

# Muovi il veicolo di una distanza x (ad esempio 50 metri)
distance = 20  # Imposta la distanza che il veicolo deve percorrere
move_vehicle_forward(distance)

# Disconnetti il veicolo
vehicle.destroy()


True